In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd 
import random 
import sys 

2026-01-30 17:16:58.292115: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df=pd.read_csv('train.csv')
text="".join(df['text'].dropna().astype(str).str.lower())
print("Length of text: {} characters".format(len(text)))

Length of text: 35680894 characters


In [3]:
vocab=sorted(set(text))
print("Number of unique characters: {}".format(len(vocab)))
char2idx={c:i for i,c in enumerate(vocab)}
idx2char=np.array(vocab)
text_as_int=np.array([char2idx[c] for c in text])
print("Text converted to integers:")
print(text_as_int[:20])
print("Characters corresponding to integers:")
print("".join(idx2char[text_as_int[:20]]))


Number of unique characters: 104
Text converted to integers:
[41 44 57 51 48 53  1  9 57 44 60 59 44 57 58 10  1 14  1 53]
Characters corresponding to integers:
berlin (reuters) - n


In [4]:
seq_length=100
char_dataset=tf.data.Dataset.from_tensor_slices(text_as_int)
sequences=char_dataset.batch(seq_length+1,drop_remainder=True)
def split_input_target(chunk):
    return chunk[:-1],chunk[1:]
dataset=sequences.map(split_input_target)
BATCH_SIZE=64
BUFFER_SIZE=10000
dataset=dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE,drop_remainder=True)

In [6]:
vocab_size=len(vocab)
embedding_dim=64
rnn_units=256
model=tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size,embedding_dim,input_shape=(None,)),
    tf.keras.layers.LSTM(rnn_units,return_sequences=True,recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
])
def loss(labels,logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels,logits,from_logits=True)
model.compile(optimizer='adam',loss=loss)
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, None, 64)       │         6,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, None, 256)      │       328,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, None, 104)      │        26,728 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 362,088 (1.38 MB)

 Trainable params: 362,088 (1.38 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
EPOCHS=20
history=model.fit(dataset,epochs=EPOCHS)

Epoch 1/20
5519/5519 ━━━━━━━━━━━━━━━━━━━━ 888s 159ms/step - loss: 1.6638
Epoch 2/20
5519/5519 ━━━━━━━━━━━━━━━━━━━━ 929s 168ms/step - loss: 1.3253
Epoch 3/20
1181/5519 ━━━━━━━━━━━━━━━━━━━━ 12:02 167ms/step - loss: 1.2837

In [ ]:
def generate_text(model,start_string,num_generate=1000,temperature=1.0):
    input_eval=[char2idx[s] for s in start_string.lower()]
    input_eval=tf.expand_dims(input_eval,0)
    text_generated=[]
    model.layers[1].reset_states()
    for _ in range(num_generate):
        predictions=model(input_eval)
        predictions=tf.squeeze(predictions,0)/temperature
        predicted_id=tf.random.categorical(predictions,num_samples=1)[-1,0].numpy()
        input_eval=tf.expand_dims([predicted_id],0)
        text_generated.append(idx2char[predicted_id])
    return start_string+"".join(text_generated)
print(generate_text(model,start_string="Once upon a time",num_generate=500,temperature=0.8))